### Instalação da biblioteca pysus

In [ ]:
pip install pysus

In [ ]:
# Seleciona os anos que serão baixados
years_for_donwload = [i for i in range(2007, 2025)]
print(years_for_donwload)

[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


In [ ]:
# Faz a importação da biblioteca e conecta com a ftp
from pysus.ftp.databases.sinan import SINAN
sinan = SINAN().load()

In [ ]:
# Caso seja necessário é possível conectar com o drive
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


### Extração e transformações iniciais

In [ ]:
import pandas as pd


# Cria a base cid-10
cid_10 = pd.read_csv('/content/CID-10_Categorias.csv')
cid_10['CAT'] = cid_10['CAT'].astype('str')

cid_10_dict = {}

for i in cid_10.values:
  cid_10_dict[i[0][:3]] = i[1]

# Função que recebe o código cid-10 e retorna a descrição
def description_cid(cid_id):
  if len(cid_id) >= 3:
    cid_clean = cid_id[:3]
    cid_description = cid_10_dict[cid_clean]

  else:
    cid_description = 'CID não registrado'

  return cid_description


# Cria a base com todos os municípios do Brasil
municipios_BR = pd.read_csv('/content/municipios_BR.csv')
municipios_BR['COD_MUN'] = municipios_BR['COD_MUN'].astype('str')

municipios_BR_dict = {}

for i in municipios_BR.values:
  municipios_BR_dict[i[0][:6]] = i[1]


# Cria a base com todos os estados do Brasil
estados_BR_dict = {
                   '11': 'Rondônia',
                   '12': 'Acre',
                   '13': 'Amazonas',
                   '14': 'Roraima',
                   '15': 'Pará',
                   '16': 'Amapá',
                   '17': 'Tocantins',
                   '21': 'Maranhão',
                   '22': 'Piauí',
                   '23': 'Ceará',
                   '24': 'Rio Grande do Norte',
                   '25': 'Paraíba',
                   '26': 'Pernambuco',
                   '27': 'Alagoas',
                   '28': 'Sergipe',
                   '29': 'Bahia',
                   '31': 'Minas Gerais',
                   '32': 'Espírito Santo',
                   '33': 'Rio de Janeiro',
                   '35': 'São Paulo',
                   '41': 'Paraná',
                   '42': 'Santa Catarina',
                   '43': 'Rio Grande do Sul',
                   '50': 'Mato Grosso do Sul',
                   '51': 'Mato Grosso',
                   '52': 'Goiás',
                   '53': 'Distrito Federal',
}


# Função que recebe um campo de data e retorna a mesma data em uma formatação amigável dia/mês/ano
def format_data(data_var):
  if len(str(data_var)) == 8:
    data_result = f'{str(data_var)[6:10]}/{str(data_var)[4:6]}/{str(data_var)[0:4]}'

  elif len(str(data_var)) == 10:
    data_result =f'{str(data_var)[8:10]}/{str(data_var)[5:7]}/{str(data_var)[0:4]}'

  else:
    data_result = '31/12/1999'

  return data_result


# Função que recebe o ano do acidente e o ano de nascimento e retorna a idade
def generate_age(ano_acidente, ano_nascimento):
  try:
    age = int(ano_acidente[6:10]) - int(ano_nascimento)

  except:
    age = 999

  return age


# Função que recebe a idade e retorna a faixa etária
def generate_age_range(age_var):

  if int(age_var) == 999:
    age_range = 'Não possui'

  elif int(age_var) < 0:
    age_range = 'Idade menor que 0'

  elif 0 <= int(age_var) <= 9:
    age_range = '0 a 9 Anos'

  elif 10 <= int(age_var) <= 19:
    age_range = '10 a 19 Anos'

  elif 20 <= int(age_var) <= 29:
    age_range = '20 a 29 Anos'

  elif 30 <= int(age_var) <= 39:
    age_range = '30 a 39 Anos'

  elif 40 <= int(age_var) <= 49:
    age_range = '40 a 49 Anos'

  elif 50 <= int(age_var) <= 59:
    age_range = '50 a 59 Anos'

  elif 60 <= int(age_var) <= 69:
    age_range = '60 a 69 Anos'

  elif 70 <= int(age_var) <= 79:
    age_range = '70 a 79 Anos'

  elif 80 <= int(age_var) <= 89:
    age_range = '80 a 89 Anos'

  elif 90 <= int(age_var):
    age_range = '90 + Anos'

  return age_range


In [ ]:
import os
import shutil
from datetime import datetime


# Cria a pasta bronze
dir_output = '/content/bronze'

if os.path.isdir(dir_output):
  shutil.rmtree(dir_output)
else:
  os.makedirs(dir_output)


# Baixa os arquivos de cada ano, aplica algumas transformações e insere na pasta bronze
for year in years_for_donwload:

  data_year = sinan.get_files(year=year)

  for file_name in data_year:

    if str(file_name)[:4] == 'ACGR':

      disease_accident = str(file_name)[:4]

      generic_disease_accident = sinan.get_files(dis_code=disease_accident, year=[year])
      download_disease_accident = sinan.download(generic_disease_accident)

      disease_accident_df = download_disease_accident.to_dataframe()

      disease_accident_df['cid_causa_acidente'] = list(map(description_cid, disease_accident_df['CID_ACID']))
      disease_accident_df['data_notificacao'] = list(map(format_data, disease_accident_df['DT_NOTIFIC']))
      disease_accident_df['data_acidente'] = list(map(format_data, disease_accident_df['DT_ACID']))
      disease_accident_df['idade'] = list(map(generate_age, disease_accident_df['data_acidente'], disease_accident_df['ANO_NASC']))
      disease_accident_df['faixa_etaria'] = list(map(generate_age_range, disease_accident_df['idade']))
      disease_accident_df['data_atendimento_medico'] = list(map(format_data, disease_accident_df['DT_ATENDE']))
      disease_accident_df['data_obito'] = list(map(format_data, disease_accident_df['DT_OBITO']))
      disease_accident_df['estado_residencia'] = disease_accident_df.SG_UF.map(estados_BR_dict)
      disease_accident_df['municipio_residencia'] = disease_accident_df.ID_MN_RESI.map(municipios_BR_dict)
      disease_accident_df['estado_notificacao'] = disease_accident_df.SG_UF_NOT.map(estados_BR_dict)
      disease_accident_df['municipio_notificacao'] = disease_accident_df.ID_MUNICIP.map(municipios_BR_dict)
      disease_accident_df['estado_empresa'] = disease_accident_df.UF_EMP.map(estados_BR_dict)
      disease_accident_df['municipio_empresa'] = disease_accident_df.MUN_EMP.map(municipios_BR_dict)
      disease_accident_df['estado_acidente'] = disease_accident_df.UF_ACID.map(estados_BR_dict)
      disease_accident_df['municipio_acidente'] = disease_accident_df.MUN_ACID.map(municipios_BR_dict)
      disease_accident_df['estado_atendimento_medico'] = disease_accident_df.UF_ATENDE.map(estados_BR_dict)

      print(f'\nNumber of Rows [{year}]: {len(disease_accident_df.axes[0])}')
      print(f'Number of Columns [{year}]: {len(disease_accident_df.axes[1])}\n')

      print(70 * '=')

      disease_accident_df.to_parquet(f'{dir_output}/data_{year}.parquet')

      original_file = f'/root/pysus/{str(generic_disease_accident)[1:9]}.parquet'

      if os.path.isfile(original_file):
        os.remove(original_file)


ACGRBR07.parquet: 100%|██████████| 20.4k/20.4k [00:01<00:00, 11.7kB/s]



Number of Rows [2007]: 19725
Number of Columns [2007]: 70



ACGRBR08.parquet: 100%|██████████| 32.4k/32.4k [00:02<00:00, 13.5kB/s]



Number of Rows [2008]: 31317
Number of Columns [2008]: 70



ACGRBR09.parquet: 100%|██████████| 36.7k/36.7k [00:02<00:00, 13.2kB/s]


...


Number of Rows [2022]: 305736
Number of Columns [2022]: 70



ACGRBR23.parquet: 100%|██████████| 460k/460k [00:36<00:00, 12.4kB/s]



Number of Rows [2023]: 444910
Number of Columns [2023]: 70



ACGRBR24.parquet: 100%|██████████| 521k/521k [00:40<00:00, 12.7kB/s]



Number of Rows [2024]: 503542
Number of Columns [2024]: 70



### Instalação do spark

In [ ]:
# Para acessar a versão mais recente: https://www.oracle.com/java/technologies/downloads/
!apt-get install openjdk-17-jdk-headless -qq > /dev/null

# Para acessar a versão mais recente: https://spark.apache.org/downloads.html
!wget -q https://dlcdn.apache.org/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz

!tar xf spark-4.0.1-bin-hadoop3.tgz

!pip install -q findspark

import os

os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-17-openjdk-amd64"

! export PATH=$JAVA_HOME/bin:$PATH

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('VISAT') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

### Consolidação dos arquivos

In [ ]:
# Lê os arquivos da bronze, consolida-os em um único arquivo e salva na silver
df = spark.read.parquet(dir_output)

silver = '/content/silver/'

df.repartition(1).write.save(silver, format='parquet')

for file in os.listdir(silver):
    if file.endswith('.parquet'):
      df_rd = spark.read.format('parquet').load(f'{silver}{file}')

print(f'Number of Rows are: {df_rd.count()}')
print(f'Number of Columns are: {len(df_rd.columns)}\n')

print(f'Schema:')
df_rd.printSchema()

Number of Rows are: 2574305
Number of Columns are: 70

Schema:
root
 |-- TP_NOT: string (nullable = true)
 |-- ID_AGRAVO: string (nullable = true)
 |-- DT_NOTIFIC: date (nullable = true)
 |-- SEM_NOT: string (nullable = true)
 |-- NU_ANO: string (nullable = true)
 |-- SG_UF_NOT: string (nullable = true)
 |-- ID_MUNICIP: string (nullable = true)
 |-- ID_REGIONA: string (nullable = true)
 |-- ID_UNIDADE: string (nullable = true)
 |-- DT_ACID: string (nullable = true)
 |-- SEM_ACID: string (nullable = true)
 |-- ANO_NASC: string (nullable = true)
 |-- NU_IDADE_N: string (nullable = true)
 |-- CS_SEXO: string (nullable = true)
 |-- CS_GESTANT: string (nullable = true)
 |-- CS_RACA: string (nullable = true)
 |-- CS_ESCOL_N: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- ID_MN_RESI: string (nullable = true)
 |-- ID_RG_RESI: string (nullable = true)
 |-- ID_PAIS: string (nullable = true)
 |-- ID_OCUPA_N: string (nullable = true)
 |-- SIT_TRAB: string (nullable = true)
 |--

### Transformações finais

In [ ]:
from pyspark.sql.functions import when, col, substring


# Lê o arquivo da silver, aplica diversas transformações e salva na gold
for file in os.listdir(silver):
    if file.endswith('.parquet'):
      df_compiled = spark.read.parquet(f'{silver}{file}')

df_compiled = df_compiled.select(col('*'),\
                   when(df_compiled.CS_SEXO == 'F', 'Feminino')\
                  .when(df_compiled.CS_SEXO == 'M', 'Masculino')\
                  .when(df_compiled.CS_SEXO == 'I', 'Ignorado')\
                  .when(df_compiled.CS_SEXO == '', 'Em branco')\
                  .when(df_compiled.CS_SEXO.isNull(), 'Em branco')\
                  .otherwise(df_compiled.CS_SEXO).alias('sexo'))\
                  \
                  .select(col('*'),\
                   when(df_compiled.CS_GESTANT == '1', '1º Trimestre')\
                  .when(df_compiled.CS_GESTANT == '2', '2º Trimestre')\
                  .when(df_compiled.CS_GESTANT == '3', '3º Trimestre')\
                  .when(df_compiled.CS_GESTANT == '4', 'Idade gestacional Ignorada')\
                  .when(df_compiled.CS_GESTANT == '5', 'Não')\
                  .when(df_compiled.CS_GESTANT == '6', 'Não se aplica')\
                  .when(df_compiled.CS_GESTANT == '9', 'Ignorado')\
                  .when(df_compiled.CS_GESTANT == '', 'Em branco')\
                  .when(df_compiled.CS_GESTANT.isNull(), 'Em branco')\
                  .otherwise(df_compiled.CS_GESTANT).alias('gestante'))\
                  \
                  .select(col('*'),\
                   when(df_compiled.CS_RACA == '1', 'Branca')\
                  .when(df_compiled.CS_RACA == '2', 'Preta')\
                  .when(df_compiled.CS_RACA == '3', 'Amarela')\
                  .when(df_compiled.CS_RACA == '4', 'Parda')\
                  .when(df_compiled.CS_RACA == '5', 'Indígena')\
                  .when(df_compiled.CS_RACA == '9', 'Ignorado')\
                  .when(df_compiled.CS_RACA == '', 'Em branco')\
                  .when(df_compiled.CS_RACA.isNull(), 'Em branco')\
                  .otherwise(df_compiled.CS_RACA).alias('raca_cor'))\
                  \
                  .select(col('*'),\
                   when(df_compiled.CS_ESCOL_N == '00', 'Analfabeto')\
                  .when(df_compiled.CS_ESCOL_N == '01', '1ª a 4ª série incompleta do EF (antigo primário ou 1º grau)')\
                  .when(df_compiled.CS_ESCOL_N == '02', '4ª série completa do EF (antigo primário ou 1º grau)')\
                  .when(df_compiled.CS_ESCOL_N == '3 ', '5ª à 8ª série incompleta do EF (antigo ginásio ou 1º grau)')\
                  .when(df_compiled.CS_ESCOL_N == '03', '5ª à 8ª série incompleta do EF (antigo ginásio ou 1º grau)')\
                  .when(df_compiled.CS_ESCOL_N == '04', 'Ensino fundamental completo (antigo ginásio ou 1º grau)')\
                  .when(df_compiled.CS_ESCOL_N == '05', 'Ensino médio incompleto (antigo colegial ou 2º grau)')\
                  .when(df_compiled.CS_ESCOL_N == '06', 'Ensino médio completo (antigo colegial ou 2º grau)')\
                  .when(df_compiled.CS_ESCOL_N == '07', 'Educação superior incompleta')\
                  .when(df_compiled.CS_ESCOL_N == '08', 'Educação superior completa')\
                  .when(df_compiled.CS_ESCOL_N == '09', 'Ignorado')\
                  .when(df_compiled.CS_ESCOL_N == '10', 'Não se aplica')\
                  .when(df_compiled.CS_ESCOL_N == '', 'Em branco')\
                  .when(df_compiled.CS_ESCOL_N.isNull(), 'Em branco')\
                  .otherwise(df_compiled.CS_ESCOL_N).alias('escolaridade'))\
                  \
                  .select(col('*'),\
                   when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '0', 'Forças Armadas, Policiais e Bombeiros Militares')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '1', 'Membros superiores do poder público, dirigentes de organizações')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '2', 'Profissionais das ciências e das artes')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '3', 'Técnicos de nível médio')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '4', 'Trabalhadores de serviços administrativos')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '5', 'Trabalhadores dos serviços, vendedores do comércio em lojas e mercados')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '6', 'Trabalhadores agropecuários, florestais, da caça e pesca')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '7', 'Trabalhadores da produção de bens e serviços industriais')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '8', 'Trabalhadores da produção de bens e serviços industriais')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '9', 'Trabalhadores de manutenção e reparação')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1) == '', 'Em branco')\
                  .when(substring(df_compiled.ID_OCUPA_N, 1, 1).isNull(), 'Em branco')\
                  .otherwise(df_compiled.ID_OCUPA_N).alias('ocupacao_grande_grupo'))\
                  \
                  .select(col('*'),\
                   when(df_compiled.SIT_TRAB == '1 ', 'Empregado registrado com carteira assinada')\
                  .when(df_compiled.SIT_TRAB == '01', 'Empregado registrado com carteira assinada')\
                  .when(df_compiled.SIT_TRAB == '02', 'Empregado não registrado')\
                  .when(df_compiled.SIT_TRAB == '03', 'Autônomo/conta própria')\
                  .when(df_compiled.SIT_TRAB == '04', 'Servidor público estatuário')\
                  .when(df_compiled.SIT_TRAB == '05', 'Servidor público celetista')\
                  .when(df_compiled.SIT_TRAB == '06', 'Aposentado')\
                  .when(df_compiled.SIT_TRAB == '07', 'Desempregado')\
                  .when(df_compiled.SIT_TRAB == '08', 'Trabalho temporário')\
                  .when(df_compiled.SIT_TRAB == '09', 'Cooperativado')\
                  .when(df_compiled.SIT_TRAB == '10', 'Trabalhador avulso')\
                  .when(df_compiled.SIT_TRAB == '11', 'Empregador')\
                  .when(df_compiled.SIT_TRAB == '12', 'Outros')\
                  .when(df_compiled.SIT_TRAB == '99', 'Ignorado')\
                  .when(df_compiled.SIT_TRAB == '', 'Em branco')\
                  .when(df_compiled.SIT_TRAB.isNull(), 'Em branco')\
                  .otherwise(df_compiled.SIT_TRAB).alias('situacao_trabalho'))\
                  \
                  .select(col('*'),\
                   when(df_compiled.TIPO_ACID == '1', 'Típico')\
                  .when(df_compiled.TIPO_ACID == '2', 'Trajeto')\
                  .when(df_compiled.TIPO_ACID == '9', 'Ignorado')\
                  .when(df_compiled.TIPO_ACID == '', 'Em branco')\
                  .when(df_compiled.TIPO_ACID.isNull(), 'Em branco')\
                  .otherwise(df_compiled.TIPO_ACID).alias('tipo_acidente'))\
                  \
                  .select(col('*'),\
                   when(df_compiled.ATENDE_MED == '1', 'Sim')\
                  .when(df_compiled.ATENDE_MED == '2', 'Não')\
                  .when(df_compiled.ATENDE_MED == '9', 'Ignorado')\
                  .when(df_compiled.ATENDE_MED == '', 'Em branco')\
                  .when(df_compiled.ATENDE_MED.isNull(), 'Em branco')\
                  .otherwise(df_compiled.ATENDE_MED).alias('atendimento_medico'))\
                   \
                  .select(col('*'),\
                   when(df_compiled.EVOLUCAO == '1', 'Cura')\
                  .when(df_compiled.EVOLUCAO == '2', 'Incapacidade temporária')\
                  .when(df_compiled.EVOLUCAO == '3', 'Incapacidade parcial permanente')\
                  .when(df_compiled.EVOLUCAO == '4', 'Incapacidade total permanente')\
                  .when(df_compiled.EVOLUCAO == '5', 'Óbito por acidente de trabalho grave')\
                  .when(df_compiled.EVOLUCAO == '6', 'Óbito por outras causas')\
                  .when(df_compiled.EVOLUCAO == '7', 'Outro')\
                  .when(df_compiled.EVOLUCAO == '8', 'Não identificado')\
                  .when(df_compiled.EVOLUCAO == '9', 'Ignorado')\
                  .when(df_compiled.EVOLUCAO == '', 'Em branco')\
                  .when(df_compiled.EVOLUCAO.isNull(), 'Em branco')\
                  .otherwise(df_compiled.EVOLUCAO).alias('evolucao_caso'))\
                   \
                  .select(col('*'),\
                   when(df_compiled.CAT == '1', 'Sim')\
                  .when(df_compiled.CAT == '2', 'Não')\
                  .when(df_compiled.CAT == '3', 'Não se aplica')\
                  .when(df_compiled.CAT == '9', 'Ignorado')\
                  .when(df_compiled.CAT == '', 'Em branco')\
                  .when(df_compiled.CAT.isNull(), 'Em branco')\
                  .otherwise(df_compiled.CAT).alias('comunicacao_acidente_trabalho'))\
                   \
                  .select(col('*'),\
                   when(df_compiled.LOCAL_ACID == '1', 'Instalações do Contratante')\
                  .when(df_compiled.LOCAL_ACID == '2', 'Via Pública')\
                  .when(df_compiled.LOCAL_ACID == '3', 'Instalações de terceiros')\
                  .when(df_compiled.LOCAL_ACID == '4', 'Domicílio próprio')\
                  .when(df_compiled.LOCAL_ACID == '9', 'Ignorado')\
                  .when(df_compiled.LOCAL_ACID == '', 'Em branco')\
                  .when(df_compiled.LOCAL_ACID.isNull(), 'Em branco')\
                  .otherwise(df_compiled.LOCAL_ACID).alias('local_acidente'))\
                   \
                  .select(col('*'),\
                   when(df_compiled.REGIME == '1', 'Hospitalar')\
                  .when(df_compiled.REGIME == '2', 'Ambulatório')\
                  .when(df_compiled.REGIME == '3', 'Ambos')\
                  .when(df_compiled.REGIME == '9', 'Ignorado')\
                  .when(df_compiled.REGIME == '', 'Em branco')\
                  .when(df_compiled.REGIME.isNull(), 'Em branco')\
                  .otherwise(df_compiled.REGIME).alias('local_tratamento'))


print(f'Number of Rows are: {df_compiled.count()}')
print(f'Number of Columns are: {len(df_compiled.columns)}\n')

print(f'Schema:')
df_compiled.printSchema()

df_compiled.repartition(1).write.save('/content/gold/', format='parquet')


Number of Rows are: 2574305
Number of Columns are: 82

Schema:
root
 |-- TP_NOT: string (nullable = true)
 |-- ID_AGRAVO: string (nullable = true)
 |-- DT_NOTIFIC: date (nullable = true)
 |-- SEM_NOT: string (nullable = true)
 |-- NU_ANO: string (nullable = true)
 |-- SG_UF_NOT: string (nullable = true)
 |-- ID_MUNICIP: string (nullable = true)
 |-- ID_REGIONA: string (nullable = true)
 |-- ID_UNIDADE: string (nullable = true)
 |-- DT_ACID: string (nullable = true)
 |-- SEM_ACID: string (nullable = true)
 |-- ANO_NASC: string (nullable = true)
 |-- NU_IDADE_N: string (nullable = true)
 |-- CS_SEXO: string (nullable = true)
 |-- CS_GESTANT: string (nullable = true)
 |-- CS_RACA: string (nullable = true)
 |-- CS_ESCOL_N: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- ID_MN_RESI: string (nullable = true)
 |-- ID_RG_RESI: string (nullable = true)
 |-- ID_PAIS: string (nullable = true)
 |-- ID_OCUPA_N: string (nullable = true)
 |-- SIT_TRAB: string (nullable = true)
 |--

### Orientações finais

Após a finalização do fluxo, baixe o arquivo que está na gold e renomeie para "ACGR_2007_2023_ouro.parquet".

Clicando no símbolo de uma pasta do lado esquerdo vão ser exibidas as pastas do projeto, clicando nos três pontinhos ao lado de cada arquivo irá ser exibida a opção de baixá-lo.